# CLP program

In [1]:
print(f'povezave: {graphs.CycleGraph(5).edges()}')
print(f'vozlisca: {graphs.CycleGraph(5).vertices()}')

povezave: [(0, 1, None), (0, 4, None), (1, 2, None), (2, 3, None), (3, 4, None)]
vozlisca: [0, 1, 2, 3, 4]


In [2]:
from sage.numerical.mip import MixedIntegerLinearProgram

# najprej bi rada definirala razdaljo - različne možnosti, glede na to kaj dobiš
# vse skupaj 4 moznosti povezava-povezava, povezava-vozlišče, vozlišče-povezava, vozlišče-vozlišče
def razdalja(moznost1, moznost2, G):
    # če imamo dve povezavi
    if isinstance(moznost1, tuple) and isinstance(moznost2, tuple):
        U, V = moznost1
        W, X = moznost2
        return min(G.distance(U, W), G.distance(U, X), G.distance(V, W), G.distance(V, X))
    # prva povezava, druga ne
    elif isinstance(moznost1, tuple):
        U, V = moznost1
        return min(G.distance(U, moznost2), G.distance(V, moznost2))
    # druga povezava, prva ne
    elif isinstance(moznost2, tuple):
        U, V = moznost2
        return min(G.distance(U, moznost1), G.distance(V, moznost1))
    # ce mamo dve vozlisci
    else:
        return G.distance(moznost1, moznost2)

In [3]:
G = graphs.CycleGraph(6)
razdalja((0, 1), 3, G)

2

In [4]:
def CLP_weak_k_dim(G, k):
    p = MixedIntegerLinearProgram(maximization=False)
    x = p.new_variable(binary=True)

    V = G.vertices()
    E = G.edges(labels=False)

    moznosti = V + E
    n = len(moznosti)

    p.set_objective(sum(x[i] for i in range(n)))

    for a in range(n):
        for b in range(a + 1, n):
            p.add_constraint(
                sum(abs(razdalja(moznosti[a], moznosti[i], G) - razdalja(moznosti[b], moznosti[i], G)) * x[i] for i in range(n)) >= k
            )

    wmdim_k = p.solve()
    mnozica_S = [moznosti[i] for i in range(n) if p.get_values(x[i]) > 0.5]

    return (wmdim_k, mnozica_S)

In [5]:
G = graphs.CycleGraph(7)
# ta graf bi imel takole: V = [0,1,2,3,4,5,6] in E = [((0,1),(1,2),(2,3),(3,4),(4,5),(5,6),(6,0)]
k = 6

print(f'{CLP_weak_k_dim(G,k)}') # - vrne S = [6, (0, 1), (1, 2), (3, 4), (4, 5)], torej zadnje vozlišče in te 4 povezave.

(14.0, [0, 1, 2, 3, 4, 5, 6, (0, 1), (0, 6), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6)])


In [6]:
def kappa_2_crti(G):
    k = 1
    while True:
        try:
            CLP_weak_k_dim(G, k)
            k += 1
        except:
            return k - 1

In [7]:
kappa_2_crti(G)

6

# CIKLI

In [8]:
def cikli_do_n(n):
    for i in range(3, n + 1):
        G = graphs.CycleGraph(i) 
        kappa_2crti = kappa_2_crti(G)
        print(f"- velikost: {i} : kappa = {kappa_2crti}")
        for k in range(1, kappa_2crti + 1):
            wmdim, _ = CLP_weak_k_dim(G, k)
            print(f"  k = {k}, wmdim = {wmdim}")

In [9]:
#G = graphs.CycleGraph(13)
#k = 6

#print(f'{CLP_weak_k_dim(G,k)}')
#print(f'{kappa_2_crti(G)}')

In [10]:
cikli_do_n(20)

- velikost: 3 : kappa = 2
  k = 1, wmdim = 3.0
  k = 2, wmdim = 6.0
- velikost: 4 : kappa = 3
  k = 1, wmdim = 3.0
  k = 2, wmdim = 4.0
  k = 3, wmdim = 8.0
- velikost: 5 : kappa = 4
  k = 1, wmdim = 3.0
  k = 2, wmdim = 5.0
  k = 3, wmdim = 8.0
  k = 4, wmdim = 10.0
- velikost: 6 : kappa = 5
  k = 1, wmdim = 3.0
  k = 2, wmdim = 4.0
  k = 3, wmdim = 6.0
  k = 4, wmdim = 9.0
  k = 5, wmdim = 12.0
- velikost: 7 : kappa = 6
  k = 1, wmdim = 3.0
  k = 2, wmdim = 5.0
  k = 3, wmdim = 7.0
  k = 4, wmdim = 10.0
  k = 5, wmdim = 12.0
  k = 6, wmdim = 14.0
- velikost: 8 : kappa = 7
  k = 1, wmdim = 3.0
  k = 2, wmdim = 4.0
  k = 3, wmdim = 6.0
  k = 4, wmdim = 8.0
  k = 5, wmdim = 11.0
  k = 6, wmdim = 14.0
  k = 7, wmdim = 16.0
- velikost: 9 : kappa = 8
  k = 1, wmdim = 3.0
  k = 2, wmdim = 5.0
  k = 3, wmdim = 7.0
  k = 4, wmdim = 9.0
  k = 5, wmdim = 12.0
  k = 6, wmdim = 14.0
  k = 7, wmdim = 16.0
  k = 8, wmdim = 18.0
- velikost: 10 : kappa = 9
  k = 1, wmdim = 3.0
  k = 2, wmdim = 4.0
  

Zgleda, da je za cikle $\kappa''(G) = n-1,$ kjer je $n$ red cikla `G = graphs.CycleGraph(n)`. Glede $wmdim_k(G)$ bi rekla, da je za $k = 1$ ta vedno enaka $3$, za višje $k$ pa zgleda, kot, da se vedno povečuje za $2$, razen, ko je $k = \lfloor \frac{velikost}{2} \rfloor + 1$. Takrat se $wmdim_k(G)$ poveča za $3$. Če bi to napisala malo bolj matematično, bi zgledalo nekako tako: $$wmdim(1) = 3,$$
$$wmdim(k) = 
\begin{cases} 
wmdim(k-1) + 2, & \text{če} \, k \neq \left\lfloor \frac{\text{velikost}}{2} \right\rfloor + 1 \\
wmdim(k-1) + 3, & \text{če} \, k = \left\lfloor \frac{\text{velikost}}{2} \right\rfloor + 1
\end{cases}$$

# Hiperkocke

In [11]:
def hiperkocke_do_n(n):
    for i in range(1, n + 1):
        G = graphs.CubeGraph(i)
        kappa_2crti = kappa_2_crti(G)
        print(f"- velikost: {i} : kappa = {kappa_2crti}")
        for k in range(kappa_2crti + 1):
            wmdim, _ = CLP_weak_k_dim(G, k)
            print(f"  k = {k}, wmdim = {wmdim}")

In [12]:
k = 1
H = graphs.CubeGraph(3)
CLP_weak_k_dim(H, k)

(3.0, ['000', '101', '110'])

In [13]:
hiperkocke_do_n(4)

- velikost: 1 : kappa = 1
  k = 0, wmdim = 0.0
  k = 1, wmdim = 2.0
- velikost: 2 : kappa = 3
  k = 0, wmdim = 0.0
  k = 1, wmdim = 3.0
  k = 2, wmdim = 4.0
  k = 3, wmdim = 8.0
- velikost: 3 : kappa = 8
  k = 0, wmdim = 0.0
  k = 1, wmdim = 3.0
  k = 2, wmdim = 4.0
  k = 3, wmdim = 7.0
  k = 4, wmdim = 8.0
  k = 5, wmdim = 12.0
  k = 6, wmdim = 14.0
  k = 7, wmdim = 17.0
  k = 8, wmdim = 20.0
- velikost: 4 : kappa = 20
  k = 0, wmdim = 0.0
  k = 1, wmdim = 4.0
  k = 2, wmdim = 5.0
  k = 3, wmdim = 7.0
  k = 4, wmdim = 8.0
  k = 5, wmdim = 11.0
  k = 6, wmdim = 13.0
  k = 7, wmdim = 15.0
  k = 8, wmdim = 16.0
  k = 9, wmdim = 20.0
  k = 10, wmdim = 22.0
  k = 11, wmdim = 24.0
  k = 12, wmdim = 27.0
  k = 13, wmdim = 30.0
  k = 14, wmdim = 32.0
  k = 15, wmdim = 35.0
  k = 16, wmdim = 38.0
  k = 17, wmdim = 40.0
  k = 18, wmdim = 43.0
  k = 19, wmdim = 46.0
  k = 20, wmdim = 48.0
